In [1]:
!pip install --upgrade --no-deps git+git://github.com/Theano/Theano.git --user

  Cloning git://github.com/Theano/Theano.git to /tmp/pip-VFotoU-build
  Found existing installation: Theano 0.9.0.dev1
    Not uninstalling theano at /home/notebook/.local/lib/python2.7/site-packages, outside environment /resources/.virtualenv/python2
  Running setup.py install for Theano ... - \ | / - \ | / - \ | / - \ | / - done


In [2]:
import theano

Couldn't import dot_parser, loading of dot files will not be possible.


In [3]:
import theano.tensor as T

### Sneak peek

##### Here is an example of how to use Theano. It doesn’t show off many of Theano’s features, but it illustrates concretely what Theano is.

In [3]:
# declare two symbolic floating-point scalars
a = T.dscalar()
b = T.dscalar()

# create a simple expression
c = a + b

# convert the expression into a callable object that takes (a,b)
# values as input and computes a value for c
f = theano.function([a,b], c)

# bind 1.5 to 'a', 2.5 to 'b', and evaluate 'c'
assert 4.0 == f(1.5, 2.5)

In [4]:
print(f(1.5,2.5))

4.0


#####Theano is not a programming language in the normal sense because you write a program in Python that builds expressions for Theano. Still it is like a programming language in the sense that you have to

  -declare variables (a,b) and give their types
  
  -build expressions for how to put those variables together
  
  -compile expression graphs to functions in order to use them for     computation.
##### It is good to think of theano.function as the interface to a compiler which builds a callable object from a purely symbolic graph. One of Theano’s most important features is that theano.function can optimize a graph and even compile some or all of it into native machine instructions.

In [14]:
#Generalizing
x = T.dscalar('x')
y = T.dscalar('y')
z = x + y
f = theano.function([x,y], z)

In [15]:
f(2,3)

array(5.0)

In [18]:
import numpy as np

In [19]:
np.allclose(f(3.5,3.5),7.0)

True

In [20]:
type(x)

theano.tensor.var.TensorVariable

In [21]:
T.dscalar

TensorType(float64, scalar)

In [23]:
x.type is T.dscalar

True

In [25]:
T.pprint(z)

'(x + y)'

In [31]:
x = T.dmatrix('x')
y = T.dmatrix('y')
z = x + y
f = theano.function([x,y],z)

In [33]:
f([[1, 2], [3, 4]], [[10, 20], [30, 40]])

array([[ 11.,  22.],
       [ 33.,  44.]])

In [35]:
T.dmatrix

TensorType(float64, matrix)

In [36]:
 f(np.array([[1, 2], [3, 4]]), np.array([[10, 20], [30, 40]]))

array([[ 11.,  22.],
       [ 33.,  44.]])

The following types are available:

byte: bscalar, bvector, bmatrix, brow, bcol, btensor3, btensor4

16-bit integers: wscalar, wvector, wmatrix, wrow, wcol, wtensor3, wtensor4

32-bit integers: iscalar, ivector, imatrix, irow, icol, itensor3, itensor4

64-bit integers: lscalar, lvector, lmatrix, lrow, lcol, ltensor3, ltensor4

float: fscalar, fvector, fmatrix, frow, fcol, ftensor3, ftensor4

double: dscalar, dvector, dmatrix, drow, dcol, dtensor3, dtensor4

complex: cscalar, cvector, cmatrix, crow, ccol, ctensor3, ctensor4

### Exercise

In [6]:
a = theano.tensor.vector() # declare variable
b = T.vector()
out = a ** 2 + b ** 2 + 2 * a * b              # build symbolic expression
f = theano.function([a, b], out)   # compile function
print(f([1, 2], [4, 5]))

[ 25.  49.]


### Logistic Function

In [4]:
x = T.dmatrix('x')
s = 1 / (1 + T.exp(-x))
logistic = theano.function([x], s)
logistic([[0, 1], [-1, -2]])

array([[ 0.5       ,  0.73105858],
       [ 0.26894142,  0.11920292]])

### Computing More than one Thing at the Same Time

In [18]:
a = T.dmatrices('a')
b = T.dmatrices('b')
diff = a - b
abs_diff = abs(diff)
diff_squared = diff**2
f = theano.function([a, b], [diff, abs_diff, diff_squared])

In [21]:
f([[1,2],[2,3]], [[1,2],[7,7]])

[array([[ 0.,  0.],
        [-5., -4.]]), array([[ 0.,  0.],
        [ 5.,  4.]]), array([[  0.,   0.],
        [ 25.,  16.]])]

### Setting a Default Value for an Argument

In [33]:
x, y = T.dscalars('x', 'y')
z = x + y
f = theano.function([x, theano.In(y, value=1)], z)

In [35]:
f(1)

array(2.0)

Inputs with default values must follow inputs without default values (like Python’s functions). There can be multiple inputs with default values. These parameters can be set positionally or by name, as in standard Python:

In [36]:
x, y, w = T.dscalars('x', 'y', 'w')
z = (x + y) * w

In [39]:
f = theano.function([x, theano.In(y, value=1), theano.In(w, value=2, name='w_by_name')], z)

In [40]:
f(1)

array(4.0)

In [41]:
f(1,2)

array(6.0)

In [42]:
f(1,2,3)

array(9.0)

In [43]:
f(1, w_by_name=3)

array(6.0)

### Using Shared Variables

It is also possible to make a function with an internal state. For example, let’s say we want to make an accumulator: at the beginning, the state is initialized to zero. Then, on each function call, the state is incremented by the function’s argument.

First let’s define the accumulator function. It adds its argument to the internal state, and returns the old state value.

In [48]:
state = theano.shared(0)
inc = T.iscalar('inc')
accumulator = theano.function([inc], state, updates=[(state, state+inc)])

In [49]:
state.get_value()

array(0)

In [50]:
accumulator(1)

array(0)

In [51]:
state.get_value()

array(1)

In [52]:
state.set_value(-1)

In [53]:
accumulator(1)

array(-1)

In [54]:
state.get_value()

array(0)

It may happen that you expressed some formula using a shared variable, but you do not want to use its value. In this case, you can use the givens parameter of function which replaces a particular node in a graph for the purpose of one particular function.

In [57]:
fn_of_state = state * 2 + inc
# The type of foo must match the shared variable we are replacing
# with the ``givens``
foo = T.scalar(dtype=state.dtype)
skip_shared = theano.function([inc, foo], fn_of_state, givens=[(state, foo)])
skip_shared(1, 3)  # we're using 3 for the state, not state.value
print(state.get_value())  # old state still there, but we didn't use it
0

0


0

##### The givens parameter can be used to replace any symbolic variable, not just a shared variable. You can replace constants, and expressions, in general. Be careful though, not to allow the expressions introduced by a givens substitution to be co-dependent, the order of substitution is not defined, so the substitutions have to work in any order.

##### In practice, a good way of thinking about the givens is as a mechanism that allows you to replace any part of your formula with a different expression that evaluates to a tensor of same shape and dtype.

### Copying Function

Theano functions can be copied, which can be useful for creating similar functions but with different shared variables or updates. This is done using the copy() method of function objects. The optimized graph of the original function is copied, so compilation only needs to be performed once.

In [4]:
state = theano.shared(0)
inc = T.iscalar('inc')
accumulator = theano.function([inc], state, updates=[(state, state+inc)])

In [6]:
accumulator(1)
accumulator(2)

array(1)

In [8]:
state.get_value()

array(3)

In [9]:
new_state = theano.shared(0)
new_accumulator = accumulator.copy(swap={state:new_state})

In [10]:
new_accumulator(20)
new_state.get_value()

array(20)

We now create a copy with updates removed using the delete_updates parameter, which is set to False by default:

In [11]:
null_accumulator = accumulator.copy(delete_updates=True)
null_accumulator(9000)
state.get_value()

array(3)

### Using Random Numbers

Because in Theano you first express everything symbolically and afterwards compile this expression to get functions, using pseudo-random numbers is not as straightforward as it is in NumPy, though also not too complicated.

The way to think about putting randomness into Theano’s computations is to put random variables in your graph. Theano will allocate a NumPy RandomStream object (a random number generator) for each such variable, and draw from it as necessary. We will call this sort of sequence of random numbers a random stream. Random streams are at their core shared variables, so the observations on shared variables hold here as well. Theanos’s random objects are defined and implemented in RandomStreams and, at a lower level, in RandomStreamsBase.

In [13]:
from theano.tensor.shared_randomstreams import RandomStreams
from theano import function
srng = RandomStreams(seed=234)
rv_u = srng.uniform((2,2))
rv_n = srng.normal((2,2))
f = function([], rv_u)
g = function([], rv_n, no_default_updates=True)    #Not updating rv_n.rng
nearly_zeros = function([], rv_u + rv_u - 2 * rv_u)

In [15]:
print(f(), f())

(array([[ 0.12672381,  0.97091597],
       [ 0.13989098,  0.88754825]]), array([[ 0.31971415,  0.47584377],
       [ 0.24129163,  0.42046081]]))


In [17]:
print(g(), g()) #same output because there's no update of the input

(array([[ 0.37328447, -0.65746672],
       [-0.36302373, -0.97484625]]), array([[ 0.37328447, -0.65746672],
       [-0.36302373, -0.97484625]]))
